# Model Selection

---
**Outline**

1. Review - Cross Validation and Model Selection
2. Cross Validation
3. Model Selection

In [2]:
# Load data manipulation package
import numpy as np
import pandas as pd
import itertools

# Load data visualization package
import matplotlib.pyplot as plt
import seaborn as sns

# Cross Validation and Model Selection

## **1. Resampling Methods**
---

#### **1.1 Validation Set Approach**

**Procedure**:
1.  Randomly divide n observations to training set and validation set or hold-out set.
2. The model is fit on the training set.
3. Asses the model performance (Deviance or Pseudo R-squared) on the validation set.

<center>
<img src="../assets/validation_set_approach.jpg" width = 500>
</center>

**Potential drawbacks**:
1. The model performance rate can be highly variable.
2. Validation set performance may tend to overestimate performance for the model fit on the entire data set.


#### **1.2 Leave-One-Out Cross-Validation (LOOCV)**

**Procedure**:
1. Split n observations into a training set containing all but one observation.
2. Validation set is that one observation.

<center>
<img src="../assets//loocv.jpg" width = 500>
</center>

3. Model performance is estimated by averaging the n resulting assessment.

$$
\text{CV}_{(n)}=\frac{1}{n}\sum_{i=1}^{n}\text{AIC}_{(i)}
$$

**Potential drawbacks**:
1. There is no randomness in the training/validation set splits;
performing LOOCV multiple times will always yield the same result.
2. Expensive to implement, since the model has to be fit n times.

#### **1.3 k-Fold Cross-Validation**

**Procedure**:
1. Randomly divide n observations into $k$ groups ($k$ folds) of approximately equal size,
typically $k=5$ or $k=10$.
2. Choose one fold as a validation set,
and the remainder $k-1$ folds as training set.

<center>
<img src="../assets/k-fold.jpg" width = 500>
</center>

3. Fit model in training set,
assess the fitted model in validation set.
Repeat step 2 and 3 until $k$ times.
4. The $k$-fold CV estimate is computed
by averaging model performance
from step 3.

$$
\text{CV}_{(k)}=\frac{1}{k}\sum_{i=1}^{k}\text{AIC}_{(i)}
$$

---

From the same data set, 10 times validation set approach produces highly variable model performance.

<center>
<img src="../assets/validation_set_approach_.jpg" width = 500>
</center>

But we can see less variability from the 10-fold CV result.

<center>
<img src="../assets/10_fold_CV.jpg" width = 500>
</center>

## **2. Variable Selection Methods**
---

#### **2.1 Best Subset Selection**
Fit a separate model for each possible combination of the $p$ predictors.

**Procedure**:
1.  Begin with fitting $M_{0}$, the null model that only  contains intercept $\beta_{0}$.
2. For $k = 1,2,\dots,p$ :
  - Fit all $\binom{p}{k}$ models that contain exactly $k$ predictors.
  - Pick the best among those models by the best CV score, and call it $M_{k}$.
3. Select a single best model from among $M_{0}, \dots, M_{p}$ by the best CV score.

**Potential drawbacks**:
- In general, there are $2^{p}$ possible predictors combination for $p$ predictors.
- So if $p = 10$ predictors, there are approximately 1,000 possible models to be fitted.
- If $p = 20$ predictors, then there are over one million possibilities!
- The number of possible models that must be considered grows rapidly as $p$ increases.
- Best subset selection becomes computationally infeasible for values of $p$ greater than around 40, even with extremely fast modern computers.

#### **2.2 Forward Selection**
Begin with null model (no predictors), then adds predictors that gives the greatest additional improvement to the model, one-at-a-time.

**Procedure**:
1.  Begin with fitting $M_{0}$, the null model that only  contains intercept $\beta_{0}$.
2. For $k = 0,\dots,p-1$ :
  - Consider all $p−k$ models that augment the predictors in $M_{k}$ with one additional predictor.
  - Choose the best among those $p−k$ models by the best CV score and call it $M_{k+1}$.
3. Select a single best model from among $M_{0}, \dots, M_{p}$ by the best CV score.

#### **2.3 Backward Elimination**
Begin with the full model containing all p predictors,
and then iteratively removes the least useful predictor, one-at-a-time.

**Procedure**:
1.  Begin with fitting $M_{p}$, the full model, which contains all $p$ predictors.
2. For $k = p,p-1\dots,1$ :
  - Consider all $k$ models that contain all but one of the predictors in $M_{k}$, for a total of $k−1$ predictors.
  - Choose the best among these $k$ models by the best CV score and call it $M_{k-1}$.
3. Select a single best model from among $M_{0}, \dots, M_{p}$ by the best CV score.

# **Load Data**
---

In [3]:
# Import dataset from csv file
data = pd.read_csv('../data/horseshoe_crab.csv')

# Table check
data.head()

,index,Color,Spine,Width,Weight,Satellite
0,0,2,3,28.3,3.05,8
1,1,3,3,26.0,2.60,4
2,2,3,3,25.6,2.15,0
3,3,4,2,21.0,1.85,0
4,4,2,3,29.0,3.00,1


In [4]:
# Information check
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      173 non-null    int64  
 1   Color      173 non-null    int64  
 2   Spine      173 non-null    int64  
 3   Width      173 non-null    float64
 4   Weight     173 non-null    float64
 5   Satellite  173 non-null    int64  
dtypes: float64(2), int64(4)
memory usage: 8.2 KB


- The dataset has **173 observations** from **5 variables**:
  - `Color` : multicategory (ordinal)
  - `Spine` : multicategory (nominal)
  - `Width` : continuous
  - `Weight` : continuous
  - `Satellite` : discrete (**response variable**)
- We gonna treat response variable `Satellite` as binary response (0 or 1).
- We need to code number of satellites into 1 for having satellites > 0, and 0 for having satellites = 0.

In [5]:
# Code the response variable Satellite
# Satellite=0 --> Satellite=0, otherwise Satellite=1
data['Satellite'] = data['Satellite'].apply(lambda x: 0 if x==0 else 1)

# Data check
data.head()

,index,Color,Spine,Width,Weight,Satellite
0,0,2,3,28.3,3.05,1
1,1,3,3,26.0,2.60,1
2,2,3,3,25.6,2.15,0
3,3,4,2,21.0,1.85,0
4,4,2,3,29.0,3.00,1


# **Cross Validation**
---

## **Data Preparation**
---

We use resampling methods to estimate Akaike Information Criterion (AIC) from model:
$$
\text{logit(satellite)} = \beta_{0} + \beta_{1}(\text{width}) + \beta_{2}(\text{weight})
$$


In [6]:
# Define X and y
X = data[['Width', 'Weight']].to_numpy()
y = data['Satellite'].to_numpy()

#### **AIC (Akaike Information Criterion)**

- AIC penalizes a model for having many parameters.
- The optimal model is the one that tends to have the maximum log likelihood.
- That is the model that minimizes:

$$
\text{AIC} = -2(\text{log likelihood - number of parameters in model})
$$
- When comparing two models, the smaller value of AIC indicates the better model.


## **1. Validation Set Approach**
---
**Procedure**:
1.  Randomly divide n observations to training set and validation set or hold-out set.
2. The model is fit on the training set.
3. Calculate AIC on the validation set.

- You can use `train_test_split` from `sklearn.model.selection` to perform validation set approach.
- Find the detail [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
- Example:
```
sklearn.model_selection.train_test_split(Parameters)
```
- Some of Parameters:
  - `test_size`   : float or int, default = `None`
  - `train_size`  : float or int, default = `None`
  - `shuffle`     : bool, default = `True`
- Returns:
  - splitting : list, length = 2*len(arrays)

For now, we will cross validate AIC score of the model with validation set approach from scratch.

Define function `validation_set_split()` to split sample data set based on validation set approach.

In [7]:
def validation_set_split(X, random_state=42):
    """
    Function to split sample with validation set approach.

    Parameters
    ----------
    X : {array-like} of shape (n_sample, n_predictors)
      All predictors set.

    random_state : int
      Pass an int for reproducible output across multiple function calls.

    Returns
    -------
    train_ind_list : list
      Contains data index of train set.

    valid_ind_list : list
      Contains data index of validation set.
    """
    # Extract sample size
    n_samples, _ = data.shape

    # Set random state (to make it replicable)
    np.random.seed(random_state)

    # Randomize index
    random_ind = np.random.choice(n_samples,
                                  size = n_samples,
                                  replace = False)

    # Split data
    # First 50% of sample --> validation set
    # The remaining is the train set
    n_half = int(n_samples / 2)
    valid_ind = random_ind[:n_half]
    train_ind = random_ind[n_half:]

    # Define index list of each set
    train_ind_list = [train_ind]
    valid_ind_list = [valid_ind]

    return train_ind_list, valid_ind_list

In [8]:
# Check the function
validation_set_index = validation_set_split(X = X,
                                            random_state = 42)

train_index, valid_index =  validation_set_index

# Print train size and valid size
print("Validation Set Approach")
print(f"n_train = {np.shape(train_index)[1]}")
print(f"n_valid = {np.shape(valid_index)[1]}")

Validation Set Approach
n_train = 87
n_valid = 86


Function `validation_set_split()` correctly divide your sample of size 173 into train set containing 87 observations and validation set containing 86 observations.

Let's check that the train and validation set indexes are random.

In [9]:
print(train_index)

[array([ 81,  79,  84,  39,  98,  86, 117, 168, 145,  47,  94, 138,  61,
        73,  33, 165, 152, 135,  62, 171, 109, 112, 105,  53,   5, 127,
         3, 160,  49,  35,  80,  77,  34,  46,   7,  43,  70, 132, 110,
        91,  83, 155, 156,  89,   8,  13,  59, 148, 131,  17,  72, 159,
       134, 144, 158,  63,  54, 107,  50, 170,  58,  48,  88,  21,  57,
       167, 129,  37, 163,   1,  52, 149, 130, 151, 103,  99, 116,  87,
        74, 121, 172,  20,  71, 106,  14,  92, 102])]


In [10]:
print(valid_index)

[array([162,  42,  90,  60, 114, 137,  41,  15, 113, 108, 124,  82,  78,
        38,  31,   9, 111,  56,  24, 153,  93,  45, 146,  29,  55,  65,
       143,  19,  16, 141,  30,  18,  12, 139, 115, 154, 136, 147,  51,
       126,  66, 119,   2,  97, 169, 150, 164,  85,  26, 157, 166, 101,
        68,  36, 133,  22, 142,  95,  67, 100,  11,  76,  75,   6,  27,
       125,   4,  32,  69, 118, 161,  10, 104, 120,   0, 140, 122,  64,
        44,  96,  28,  40, 123,  25,  23, 128])]


In [11]:
def AIC(y_true, y_pred, p):
    """
    Function to split sample with validation set approach.

    Parameters
    ----------
    y_true : {array-like} of shape (n_sample, )
      Actual value of response variable.

    y_pred : {array-like} of shape (n_sample, 1)
      The success probability of X.

    p : int
      Number of parameters in model.

    Returns
    -------
    aic : float
      AIC value.
    """
    # Find the average log likelihood value
    llf = np.mean(y_true*np.log(y_pred) + (1-y_true)*np.log(1-y_pred))

    # AIC value is sensitive to number of parameters
    # The average log likelihood represent value for 1 unit observation
    # AIC from average llf is not comparable
    # Multiply llf by n_sample=173 to make its AIC comparable
    llf *= 173

    # Calculate AIC
    aic = -2 * (llf - p)

    return aic

Next, define function `AIC()` to calculate AIC score of fitted model on validation set.

**Why is the llf above is multiplied by 173?**
- Note that **AIC** value **from n_train** observations is not comparable, since n_train of one CV method with another can be different.
- **We can't compare** that AIC value with other AIC value from model with different number of observations, say we want to compare AIC from validation set (n_valid = 87 observations) with AIC from full sample (n_sample = 173 observations).
- Hence, **to make AIC values comparable**, we calculate AIC from log-likelihood value from the same number of observations, we use **n_sample = 173 observations**.

Next, define function `cross_validate()` to fit a model on train set and calculate its AIC from validation set.

We use package from `Statsmodels` to calculate `y_pred`.

In [12]:
# Load the pakage
import statsmodels.api as sm

def cross_validate(X, y, method, cv, random_state=42):
    """
    Function to evaluate AIC by cross-validation method.

    Parameters
    ----------
    X : {array-like} of shape (n_sample, n_predictors)
      The independent variable or predictors.

    y : {array-like} of shape (n_sample, )
      The dependent or response variable.

    method : cross-validation splitter
      Cross-validation method.

    cv : int
      Number of folds for k-Fold CV.

    random_state : int, default=42
      Pass an int for reproducible output across multiple function calls.

    Returns
    -------
    score : float
      The average AIC score.
    """
    # Split train and valid set based on CV method
    if method == "validation_set":
        train_ind_list, valid_ind_list = validation_set_split(X = X,
                                                              random_state = random_state)
    elif method == "loocv":
        train_ind_list, valid_ind_list = loocv_split(X = X)
    elif method == "kfold":
        train_ind_list, valid_ind_list = kfold_split(X = X,
                                                     k = cv,
                                                     random_state = random_state)

    # Define the number of train sets
    n_split = len(train_ind_list)

    # Initialize AIC score list for each valid set
    score_list = []

    for i in range(n_split):
        # Extract data from index
        X_train = X[train_ind_list[i]]
        y_train = y[train_ind_list[i]]
        X_valid = X[valid_ind_list[i]]
        y_valid = y[valid_ind_list[i]]

        # Add constant
        X_train = sm.add_constant(X_train,
                                  has_constant = 'add')
        X_valid = sm.add_constant(X_valid,
                                  has_constant = 'add')

        # Fitting model
        model = sm.Logit(y_train, X_train)
        results = model.fit(disp = False)

        # Calculate success probability
        y_pred_train = results.predict(X_train)
        y_pred_valid = results.predict(X_valid)

        # Calculate AIC
        aic_train = AIC(y_true = y_train,
                        y_pred = y_pred_train,
                        p = X_train.shape[1])
        aic_valid = AIC(y_true = y_valid,
                        y_pred = y_pred_valid,
                        p = X_train.shape[1])

        # Append AIC score in list
        score_list.append(aic_valid)

    # Calculate CV Score
    score = np.mean(score_list)

    return score

In [13]:
# Check the function
validation_set_score = cross_validate(X = X,
                                      y = y,
                                      method = 'validation_set',
                                      cv = None,
                                      random_state = 42)

validation_set_score

218.97299221422074

## **2. LOOCV**
---
**Procedure**:
1. Split n observations into a training set containing all but one observation.
2. Validation set is that one observation.
3. Model performance is estimated by averaging the n resulting assessment.

$$
\text{CV}_{(n)}=\frac{1}{n}\sum_{i=1}^{n}\text{AIC}_{(i)}
$$

- You can use `cross_validate` from `sklearn.model.selection` to perform cross-validation.
- Find the details [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)
- Example:
```
sklearn.model_selection.cross_validate(Parameters)
```
- Some of Parameters:
  - `estimator` : estimator object implementing ‘fit’ from sklearn
  - `X` : array-like of shape (n_samples, n_features)
  - `y` : array-like of shape (n_samples,)
  - `scoring` : find [here](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter), default=`None`
  - `cv` : int, cross-validation generator or an iterable, default=`None` to use the default 5-fold cross validation.
    - You can define other method for parameter `cv`
    - Here we use method `LeaveOneOut()` from `sklearn.model.selection`.
    - Find the details [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html).
    - Note: `LeaveOneOut()` is equivalent to `KFold(n_splits=n)`
- Returns:
  - scores : dict of float arrays of shape (n_splits,)

For now, we will cross validate AIC score of the model from scratch.

Define function `loocv_split()` to split sample data set based on Leave-One-Out CV.

In [14]:
def loocv_split(X):
    """
    Function to split sample with leave-one-out cv method.

    Parameters
    ----------
    X : {array-like} of shape (n_sample, n_predictors)
      All predictors set.

    Returns
    -------
    train_ind_list : list
      Contains data index of train set.

    valid_ind_list : list
      Contains data index of validation set.
    """
    # Extract sample size
    n_samples, _ = X.shape

    # Define initial list for train and valid index
    train_ind_list = []
    valid_ind_list = []

    # Define the number of train and valid sets
    # There are n_sample train and valid set
    list_ind = [i for i in range(n_samples)]

    # Define train and valid index for each set
    for i in range(n_samples):
        valid_ind = [i]
        train_ind = [p for p in list_ind if p not in valid_ind]

        # Append train and valid index in list
        valid_ind_list.append(valid_ind)
        train_ind_list.append(train_ind)

    return train_ind_list, valid_ind_list

In [15]:
# Check the function
loocv_index = loocv_split(X = X)

train_index, valid_index =  loocv_index

# Print train size and valid size
print("LOOCV for each set from n_sample = 173")
print(f"n_train = {np.shape(train_index)[1]}")
print(f"n_valid = {np.shape(valid_index)[1]}")

LOOCV for each set from n_sample = 173
n_train = 172
n_valid = 1


Function `loocv_split()` correctly divide your sample of size 173.

Each sample is used once as a test set (`n_valid = 1`) while the remaining samples form the training set (`n_train = 172`).

In [16]:
print(np.array(train_index))

[[  1   2   3 ... 170 171 172]
 [  0   2   3 ... 170 171 172]
 [  0   1   3 ... 170 171 172]
 ...
 [  0   1   2 ... 169 171 172]
 [  0   1   2 ... 169 170 172]
 [  0   1   2 ... 169 170 171]]


In [17]:
print(np.array(valid_index))

[[  0]
 [  1]
 [  2]
 [  3]
 [  4]
 [  5]
 [  6]
 [  7]
 [  8]
 [  9]
 [ 10]
 [ 11]
 [ 12]
 [ 13]
 [ 14]
 [ 15]
 [ 16]
 [ 17]
 [ 18]
 [ 19]
 [ 20]
 [ 21]
 [ 22]
 [ 23]
 [ 24]
 [ 25]
 [ 26]
 [ 27]
 [ 28]
 [ 29]
 [ 30]
 [ 31]
 [ 32]
 [ 33]
 [ 34]
 [ 35]
 [ 36]
 [ 37]
 [ 38]
 [ 39]
 [ 40]
 [ 41]
 [ 42]
 [ 43]
 [ 44]
 [ 45]
 [ 46]
 [ 47]
 [ 48]
 [ 49]
 [ 50]
 [ 51]
 [ 52]
 [ 53]
 [ 54]
 [ 55]
 [ 56]
 [ 57]
 [ 58]
 [ 59]
 [ 60]
 [ 61]
 [ 62]
 [ 63]
 [ 64]
 [ 65]
 [ 66]
 [ 67]
 [ 68]
 [ 69]
 [ 70]
 [ 71]
 [ 72]
 [ 73]
 [ 74]
 [ 75]
 [ 76]
 [ 77]
 [ 78]
 [ 79]
 [ 80]
 [ 81]
 [ 82]
 [ 83]
 [ 84]
 [ 85]
 [ 86]
 [ 87]
 [ 88]
 [ 89]
 [ 90]
 [ 91]
 [ 92]
 [ 93]
 [ 94]
 [ 95]
 [ 96]
 [ 97]
 [ 98]
 [ 99]
 [100]
 [101]
 [102]
 [103]
 [104]
 [105]
 [106]
 [107]
 [108]
 [109]
 [110]
 [111]
 [112]
 [113]
 [114]
 [115]
 [116]
 [117]
 [118]
 [119]
 [120]
 [121]
 [122]
 [123]
 [124]
 [125]
 [126]
 [127]
 [128]
 [129]
 [130]
 [131]
 [132]
 [133]
 [134]
 [135]
 [136]
 [137]
 [138]
 [139]
 [140]
 [141]
 [142]

Calculate AIC score with `cross_validate()` function.

In [18]:
# Calculate CV score
loocv_score = cross_validate(X = X,
                             y = y,
                             method = 'loocv',
                             cv = None,
                             random_state = 42)

loocv_score

204.72220473824714

## **3. k-Fold CV**
---
**Procedure**:
1. Randomly divide n observations into $k$ groups ($k$ folds) of approximately equal size,
typically $k=5$ or $k=10$.
2. Choose one fold as a validation set,
and the remainder $k-1$ folds as training set.
3. Fit model in training set,
assess the fitted model in validation set.
Repeat step 2 and 3 until $k$ times.
4. The $k$-fold CV estimate is computed
by averaging model performance
from step 3.

$$
\text{CV}_{(k)}=\frac{1}{k}\sum_{i=1}^{k}\text{AIC}_{(i)}
$$

- You can use `cross_validate` from `sklearn.model.selection` to perform k-Fold cross-validation.
- Find the details [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)
- Example:
```
sklearn.model_selection.cross_validate(Parameters)
```
- Some of Parameters:
  - `estimator` : estimator object implementing ‘fit’ from sklearn
  - `X` : array-like of shape (n_samples, n_features)
  - `y` : array-like of shape (n_samples,)
  - `scoring` : find [here](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter), default=`None`
  - `cv` : int, cross-validation generator or an iterable, default=`None` to use the default 5-fold cross validation.
    - You can define other method for parameter `cv`
    - Here we use method `kFold` from `sklearn.model.selection`.
    - Find the details [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).
    - Example:
    ```
    KFold(n_splits=5, shuffle=False, random_state=None)
    ```
- Returns:
  - scores : dict of float arrays of shape (n_splits,)



For now, we will cross validate AIC score of the model from scratch.

Define function `kfold_split()` to split sample data set based on k-Fold CV.

In [19]:
def kfold_split(X, k=5, random_state=42):
    """
    Function to split sample with validation set approach.

    Parameters
    ----------
    X : {array-like} of shape (n_sample, n_predictors)
      All predictors set.

    k : int, default = 5
      Number of folds.

    random_state : int
      Pass an int for reproducible output across multiple function calls.

    Returns
    -------
    train_ind_list : list
      Contains data index of train set.

    valid_ind_list : list
      Contains data index of validation set.
    """
    # Extract sample size
    n_samples, _ = data.shape

    # Set random state
    np.random.seed(random_state)

    # # Randomize index
    random_ind = np.random.choice(n_samples,
                                  size = n_samples,
                                  replace = False)

    # Calculate size of each fold
    fold_sizes = np.ones(k, dtype=int) * (n_samples//k)
    fold_sizes[:n_samples%k] += 1

    # Define initial list for each train and valid index
    train_ind_list = []
    valid_ind_list = []

    # Split sample
    current_ind = 0
    for size in fold_sizes:
        # Define index
        start_ind = current_ind
        end_ind = current_ind + size

        # Slice valid set
        # One fold for valid set, the remaining for train set
        valid_ind = random_ind[start_ind:end_ind]
        train_ind = np.concatenate((random_ind[:start_ind],
                                    random_ind[end_ind:]))

        # Update current index
        current_ind = end_ind

        # Append train and valid index in list
        train_ind_list.append(train_ind)
        valid_ind_list.append(valid_ind)

    return train_ind_list, valid_ind_list

In [20]:
# Check the function
kfold_index = kfold_split(X = X,
                          k = 10,
                          random_state = 42)

train_index, valid_index =  kfold_index

# Print train size and valid size
print("10-Fold CV for each set from k_sample set")
print(f"n_train = {len(train_index[0])}")
print(f"n_valid = {len(valid_index[0])}")

10-Fold CV for each set from k_sample set
n_train = 155
n_valid = 18


Function `kfold_split()` correctly divide your sample of size 173 into 10 folds.

Each fold is used once as a test set (`n_valid = 18`) while the remaining folds form the training set (`n_train = 155`).

Calculate AIC score with `cross_validate()` function.

In [21]:
# Calculate CV score
kfold_score = cross_validate(X = X,
                             y = y,
                             method = 'kfold',
                             cv = 10,
                             random_state = 42)

kfold_score

203.0843746327386

## **CV Summary**
---
Table below summarize the CV Score (AIC) from Model 2 using three methods of cross validation.

In [22]:
# Define three methods of CV in method_list
method_list = ["validation_set", "loocv", "kfold"]

# Create initial list for AIC score of each method
score_list = []

# Calculate AIC score for each method in list
for method in method_list:
    score = cross_validate(X = X,
                           y = y,
                           method = method,
                           cv = 10,
                           random_state = 42)
    score_list.append(score)

# Tabulate each method and its AIC score
summary_df = pd.DataFrame({"method": method_list,
                           "AIC Valid": score_list})

summary_df

,method,AIC Valid
0,validation_set,218.972992
1,loocv,204.722205
2,kfold,203.084375


# **Model Selection**
---

## **Data Preparation**
---

In [23]:
# Data check
data.head()

,index,Color,Spine,Width,Weight,Satellite
0,0,2,3,28.3,3.05,1
1,1,3,3,26.0,2.60,1
2,2,3,3,25.6,2.15,0
3,3,4,2,21.0,1.85,0
4,4,2,3,29.0,3.00,1


We will use all 4 predictors in model selection.
- Color : ordinal (treated as continuous)
- Spine : multicategory
- Width : continuous
- Weight : continuous

We have one multicategory predictor Spine, we need dummies for Spine, where:
- Spine = 1 as the reference
- Dummies for:
  - Spine_2
  - Spine_3

In [24]:
# Create dummy variables for Spine_2 and Spine_3
data[['Spine_2', 'Spine_3']] = pd.get_dummies(data['Spine'],
                                              dtype = 'int',
                                              drop_first = True)
data.head()

,index,Color,Spine,Width,Weight,Satellite,Spine_2,Spine_3
0,0,2,3,28.3,3.05,1,0,1
1,1,3,3,26.0,2.60,1,0,1
2,2,3,3,25.6,2.15,0,0,1
3,3,4,2,21.0,1.85,0,1,0
4,4,2,3,29.0,3.00,1,0,1


We will use **5 predictors** for model selection.
- Width : continuous
- Weight : continuous
- Color : continuous (ordinal)
- Dummy Spine_2 : category
- Dummy Spine_3 : category

In [25]:
# Define predictors X and response y
X = data[['Width', 'Weight', 'Color', 'Spine_2', 'Spine_3']].to_numpy()
y = data['Satellite'].to_numpy()

X[0:5]

array([[28.3 ,  3.05,  2.  ,  0.  ,  1.  ],
       [26.  ,  2.6 ,  3.  ,  0.  ,  1.  ],
       [25.6 ,  2.15,  3.  ,  0.  ,  1.  ],
       [21.  ,  1.85,  4.  ,  1.  ,  0.  ],
       [29.  ,  3.  ,  2.  ,  0.  ,  1.  ]])

## **1. Best Subset Selection**
---
Fit a separate model for each possible combination of the $p$ predictors.

Define function `best_subset()` to fit a model on train set and calculate its AIC from test set.

In [26]:
def best_subset(X, y, k, predictors, method, cv=5, random_state=42):
    """
    Function to perform best subset selection procedure.

    Parameters
    ----------
    X : {array-like} of shape (n_sample, n_predictors)
      All predictors set.

    y : {array-like} of shape (n_sample, )
      The dependent or response variable.

    k : int
      Number of predictors included in model.

    predictors : {array-like} of shape (n_sample, )
      Index of predictors

    method : cross-validation splitter
      Cross-validation method.

    cv : int, default=5
      Number of folds for k-Fold CV.

    random_state : int, default=42
      Pass an int for reproducible output across multiple function calls.

    Returns
    -------
    models : {array-like} of shape (n_combinations, k)
      Summary of predictors and its AIC score for each possible combination.

    best_model : {array-like} of shape (2, )
      Best model of models with the smallest AIC score.
    """
    # Initialize list of results
    results = []

    # Define sample size and  number of all predictors
    n_samples, n_predictors = X.shape

    # Initialize list of predictors and its CV Score
    pred_list = []
    score_list = []

    # Iteratively cross validate each possible combination of k
    for combi in itertools.combinations(predictors, k):
        # Extract predictors combination
        X_ = X[:, combi]
        y_ = y

        # Cross validate to get CV Score
        score_ = cross_validate(X = X_,
                                y = y_,
                                method = method,
                                cv = cv,
                                random_state = random_state)

        # Append predictors combination and its CV Score to the list
        pred_list.append(list(combi))
        score_list.append(score_)

    # Tabulate the results
    models = pd.DataFrame({"Predictors": pred_list,
                            "AIC": score_list})

    # Choose the best model of k predictors
    best_model = models.loc[models['AIC'].argmin()]

    return models, best_model

Let's check the function for `k=2` predictors included in model.

In [27]:
# Define all predictors
n_predictors = X.shape[1]
predictors = np.arange(n_predictors)

# Perform best subset selection for k=2 predictors in model
models_2, best_model_2 = best_subset(X = X,
                                     y = y,
                                     k = 2,
                                     predictors = predictors,
                                     method = 'kfold',
                                     cv = 10,
                                     random_state = 42)

In [28]:
# Print results of each possible combination
models_2

,Predictors,AIC
0,"[0, 1]",203.084375
1,"[0, 2]",199.810140
2,"[0, 3]",205.807050
3,"[0, 4]",204.926291
4,"[1, 2]",200.691316
5,"[1, 3]",206.889344
6,"[1, 4]",205.945171
7,"[2, 3]",222.548089
8,"[2, 4]",221.674235
9,"[3, 4]",234.627975


Including 2 of 5 predictors in model yields 10 possible combinations since $C(5,2)=10$.

In [29]:
# Print the best model of 2 predictors included in model
# The best model is the model with smallest AIC
best_model_2

Predictors       [0, 2]
AIC           199.81014
Name: 1, dtype: object

For `k=2` predictors in model, the best model is the model with predictors with index [0, 2] in X, which is variable Width and Color, with AIC value of 199.81.

Next, perform the best subset model selection procedure for all possible combinations in each `k` predictors included in model.

In [30]:
# Create table for the best model of each k predictors
best_subset_models = pd.DataFrame(columns = ["Predictors", "AIC"])

# Define all predictors
n_predictors = X.shape[1]
predictors = np.arange(n_predictors)

# Perform best subset selection procedure
for k in range(n_predictors+1):
    _, best_model = best_subset(X = X,
                                y = y,
                                k = k,
                                predictors = predictors,
                                method = 'kfold',
                                cv = 10,
                                random_state = 42)

    # Tabulate the best model of each k predictors
    best_subset_models.loc[k] = best_model

In [31]:
# Print the best model of each k predictors
best_subset_models

,Predictors,AIC
0,[],229.286353
1,[0],201.540786
2,"[0, 2]",199.81014
3,"[0, 1, 2]",201.616479
4,"[0, 1, 2, 4]",203.771348
5,"[0, 1, 2, 3, 4]",207.266894


Conclusion:

> - The smallest AIC score from 10-fold cross-validation is 199.81 for model with `k=2` predictors.
> - Thus, with best subset selection procedure, the best logistic regression model for horseshoe crab sample is model with 2 predictors, which is the model with predictors Width and Color.



In [32]:
# Define column index of best predictors
best_predictors = best_subset_models.loc[2]['Predictors']

# Define X with best predictors
X_best = X[:, [0,2]]

# Fit best model
X_best = sm.add_constant(X_best)
best_model = sm.Logit(y, X_best)
best_model_result = best_model.fit()

print(best_model_result.summary())

Optimization terminated successfully.
         Current function value: 0.546593
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  173
Model:                          Logit   Df Residuals:                      170
Method:                           MLE   Df Model:                            2
Date:                Fri, 08 Nov 2024   Pseudo R-squ.:                  0.1623
Time:                        04:06:32   Log-Likelihood:                -94.561
converged:                       True   LL-Null:                       -112.88
Covariance Type:            nonrobust   LLR p-value:                 1.107e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -10.0708      2.807     -3.588      0.000     -15.572      -4.569
x1             0.4583      0.

## **2. Forward Selection**
---
Begin with null model (no predictors), then adds predictors that gives the greatest additional improvement to the model, one-at-a-time.

Define function `forward()` to fit a model on train set and calculate its AIC from test set.

In [33]:
def forward(X, y, predictors, method, cv=5, random_state=42):
    """
    Function to perform best subset selection procedure.

    Parameters
    ----------
    X : {array-like} of shape (n_sample, n_predictors)
      All predictors set.

    y : {array-like} of shape (n_sample, )
      The dependent or response variable.

    predictors : {array-like} of shape (n_sample, )
      Index of predictors

    method : cross-validation splitter
      Cross-validation method.

    cv : int, default=5
      Number of folds for k-Fold CV.

    random_state : int, default=42
      Pass an int for reproducible output across multiple function calls.

    Returns
    -------
    models : {array-like} of shape (n_combinations, k)
      Summary of predictors and its AIC score for each possible combination.

    best_model : {array-like} of shape (2, )
      Best model of models with the smallest AIC score.
    """

    # Initialize list of results
    results = []

    # Define sample size and  number of all predictors
    n_samples, n_predictors = X.shape

    # Define list of all predictors
    col_list = np.arange(n_predictors)

    # Define remaining predictors for each k
    remaining_predictors = [p for p in col_list if p not in predictors]

    # Initialize list of predictors and its CV Score
    pred_list = []
    score_list = []

    # Cross validate each possible combination of remaining predictors
    for p in remaining_predictors:
        combi = predictors + [p]

        # Extract predictors combination
        X_ = X[:, combi]
        y_ = y

        # Cross validate to get CV Score
        score_ = cross_validate(X = X_,
                                y = y_,
                                method = method,
                                cv = cv,
                                random_state = random_state)

        # Append predictors combination and its CV Score to the list
        pred_list.append(list(combi))
        score_list.append(score_)

    # Tabulate the results
    models = pd.DataFrame({"Predictors": pred_list,
                            "AIC": score_list})

    # Choose the best model
    best_model = models.loc[models['AIC'].argmin()]

    return models, best_model

Perform the forward selection procedure for all possible combinations in each `k` predictors included in model.

In [34]:
# Fit null model
predictor = []
score_ = cross_validate(X = X[:,predictor],
                        y = y,
                        method = 'kfold',
                        cv = 10,
                        random_state = 42)

# Create table for the best model of each k predictors
# Append the results of null model
forward_models = pd.DataFrame({"Predictors": [predictor],
                               "AIC": [score_]})

# Define list of predictors
predictors = []
n_predictors = X.shape[1]

# Perform forward selection procedure for k=1,...,5 predictors
for k in range(n_predictors):
    _, best_model = forward(X = X,
                            y = y,
                            predictors = predictors,
                            method = 'kfold',
                            cv = 10,
                            random_state = 42)

    # Tabulate the best model of each k predictors
    forward_models.loc[k+1] = best_model
    predictors = best_model['Predictors']

In [35]:
# Print the best model of each k predictors
forward_models

,Predictors,AIC
0,[],229.286353
1,[0],201.540786
2,"[0, 2]",199.810140
3,"[0, 2, 1]",201.616479
4,"[0, 2, 1, 4]",203.771348
5,"[0, 2, 1, 4, 3]",207.266894


Conclusion:

> - The smallest AIC score from 10-fold cross-validation is 199.81 for model with `k=2` predictors.
> - Thus, with forward selection procedure, the best logistic regression model for horseshoe crab sample is model with 2 predictors, which is the model with predictors Width and Color.

In [36]:
# Define column index of best predictors
best_predictors = forward_models.loc[2]['Predictors']

# Define X with best predictors
X_best = X[:, [0,2]]

# Fit best model
X_best = sm.add_constant(X_best)
best_model = sm.Logit(y, X_best)
best_model_result = best_model.fit()

print(best_model_result.summary())

Optimization terminated successfully.
         Current function value: 0.546593
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  173
Model:                          Logit   Df Residuals:                      170
Method:                           MLE   Df Model:                            2
Date:                Fri, 08 Nov 2024   Pseudo R-squ.:                  0.1623
Time:                        04:07:31   Log-Likelihood:                -94.561
converged:                       True   LL-Null:                       -112.88
Covariance Type:            nonrobust   LLR p-value:                 1.107e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -10.0708      2.807     -3.588      0.000     -15.572      -4.569
x1             0.4583      0.

## **3. Backward Elimination**
---
Begin with the full model containing all p predictors,
and then iteratively removes the least useful predictor, one-at-a-time.

Define function `backward()` to fit a model on train set and calculate its AIC from test set.

In [37]:
def backward(X, y, predictors, method, cv=5, random_state=42):
    """
    Function to perform best subset selection procedure.

    Parameters
    ----------
    X : {array-like} of shape (n_sample, n_predictors)
      All predictors set.

    y : {array-like} of shape (n_sample, )
      The dependent or response variable.

    predictors : {array-like} of shape (n_sample, )
      Index of predictors

    method : cross-validation splitter
      Cross-validation method.

    cv : int, default=5
      Number of folds for k-Fold CV.

    random_state : int, default=42
      Pass an int for reproducible output across multiple function calls.

    Returns
    -------
    models : {array-like} of shape (n_combinations, k)
      Summary of predictors and its AIC score for each possible combination.

    best_model : {array-like} of shape (2, )
      Best model of models with the smallest AIC score.
    """
    # Initialize list of results
    results = []

    # Initialize list of predictors and its CV Score
    pred_list = []
    score_list = []

    # Cross validate each possible combination of remaining predictors
    for combi in itertools.combinations(predictors, len(predictors)-1):

        # Extract predictors combination
        X_ = X[:, combi]
        y_ = y

        # Cross validate to get CV Score
        score_ = cross_validate(X = X_,
                                y = y_,
                                method = method,
                                cv = cv,
                                random_state = random_state)

        # Append predictors combination and its CV Score to the list
        pred_list.append(list(combi))
        score_list.append(score_)

    # Tabulate the results
    models = pd.DataFrame({"Predictors": pred_list,
                            "AIC": score_list})

    # Choose the best model
    best_model = models.loc[models['AIC'].argmin()]

    return models, best_model

Perform the backward elimination procedure for all possible combinations of predictors.

In [38]:
# Fit full model
n_predictors = X.shape[1]
predictors = np.arange(n_predictors)
score_ = cross_validate(X = X[:, predictors],
                        y = y,
                        method = 'kfold',
                        cv = 10,
                        random_state = 42)

# Create table for the best model of each combination of predictors
# Append the results of the full model
backward_models = pd.DataFrame({"Predictors": [predictors],
                               "AIC": [score_]})

# Define initial k=1, to eliminate 1 predictor
k=1

# Perform backward eliminiation procedure
while(len(predictors) > 0):
    _, best_model = backward(X = X,
                             y = y,
                             predictors = predictors,
                             method = 'kfold',
                             cv = 10,
                             random_state = 42)

    # Tabulate the best model of each combination of predictors
    backward_models.loc[k] = best_model
    predictors = best_model["Predictors"]

    # Define next iteration
    # Iteratively eliminate 1 predictor
    k += 1

In [39]:
# Print the best model of each combination of predictors
backward_models

,Predictors,AIC
0,"[0, 1, 2, 3, 4]",207.266894
1,"[0, 1, 2, 4]",203.771348
2,"[0, 1, 2]",201.616479
3,"[0, 2]",199.810140
4,[0],201.540786
5,[],229.286353


Conclusion:

> - The smallest AIC score from 10-fold cross-validation is 199.81 for model with 2 predictors.
> - Thus, with backward elimination procedure, the best logistic regression model for horseshoe crab sample is model with 2 predictors, which is the model with predictors Width and Color.

In [40]:
# Define column index of best predictors
best_predictors = backward_models.loc[2]['Predictors']

# Define X with best predictors
X_best = X[:, [0,2]]

# Fit best model
X_best = sm.add_constant(X_best)
best_model = sm.Logit(y, X_best)
best_model_result = best_model.fit()

print(best_model_result.summary())

Optimization terminated successfully.
         Current function value: 0.546593
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  173
Model:                          Logit   Df Residuals:                      170
Method:                           MLE   Df Model:                            2
Date:                Fri, 08 Nov 2024   Pseudo R-squ.:                  0.1623
Time:                        04:08:31   Log-Likelihood:                -94.561
converged:                       True   LL-Null:                       -112.88
Covariance Type:            nonrobust   LLR p-value:                 1.107e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -10.0708      2.807     -3.588      0.000     -15.572      -4.569
x1             0.4583      0.

## **Model Selection Summary**
---

All three model selection methods yield the same conclusion of which combination of predictors yields the smallest AIC score.

**Best Subset Selection Advantage**
- Simple procedure.
- It yields a larger search space, which means a higher chance of finding the best model.

**Best Subset Selection Drawbacks**
- Can't be applied with very large $p$ (number of predictors), since **we have to fit $2^{p}$ models**.
- It may also suffer from statistical problems when $p$ is large.
  - The larger the search space, the higher the chance of finding models that look good on the training data, even though they might not have any predictive power on future data.

**Stepwise Methods Advantage**
- The only viable subset method when $p$ is very large.
- Forward selection can be used even when $n < p$.

**Stepwise Methods Drawbacks**
- Backward selection requires that sample size $n$ is larger than number of predictors $p$ (so that the full model can be fit).
- Stepwise methods is **not guaranteed to yield the best
model** containing a subset of the $p$ predictors.